# Replication of the optimal strategy on other node


The idea is that nodes have specifities due to their location (whether they are close to a specific type of generator or node).

Therefore, we study how would be the asset profits on other nodes.


In [1]:
import os
import requests
from datetime import datetime, timedelta
from glob import glob
import pandas as pd
import csv
import numpy as np
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d 

## Wind node

In [ ]:
node_RT_wind_data = pd.read_csv("../data/Node_RT_wind.csv")
node_DA_wind_data = pd.read_csv("../data/Node_DA_wind.csv")

In [11]:
node_RT_wind = node_RT_wind_data[node_RT_wind_data['VALUE'] == 'LMP']
node_RT_wind['date'] = pd.to_datetime(node_RT_wind['MARKET_DAY']) 
node_RT_wind['Month'] = node_RT_wind['date'].dt.month
node_DA_wind = node_DA_wind_data
node_DA_wind['date'] = pd.to_datetime(node_DA_wind['MARKET_DAY'])
node_DA_wind['Month'] = node_DA_wind['date'].dt.month
node_DA_wind.drop(columns=["source_zip"], inplace=True)

In [12]:
node_RT_wind.head()

,MARKET_DAY,NODE,TYPE,VALUE,HE1,HE2,HE3,HE4,HE5,HE6,...,HE17,HE18,HE19,HE20,HE21,HE22,HE23,HE24,date,Month
0,10/1/2022,MEC.PPWIND,Gennode,LMP,21.37,17.56,22.91,31.33,36.50,39.35,...,44.05,46.74,48.06,41.50,26.21,30.87,18.47,21.15,2022-10-01,10
1,10/2/2022,MEC.PPWIND,Gennode,LMP,11.11,21.37,21.32,21.26,21.89,24.57,...,43.97,52.69,47.13,40.94,25.95,9.96,2.44,1.35,2022-10-02,10
2,10/3/2022,MEC.PPWIND,Gennode,LMP,0.81,-1.01,-0.14,1.31,3.85,15.82,...,47.89,59.5,118.27,48.45,33.04,40.65,31.69,19.47,2022-10-03,10
3,10/4/2022,MEC.PPWIND,Gennode,LMP,27.73,11.53,28.30,20.85,17.01,12.86,...,54.57,65.65,59.43,109.99,45.46,42.85,44.65,44.10,2022-10-04,10
4,10/5/2022,MEC.PPWIND,Gennode,LMP,38.71,35.19,32.06,35.00,39.91,64.11,...,58.5,61.87,60.54,80.03,108.18,46.30,42.87,38.98,2022-10-05,10


In [13]:
def monthly_stats(df):
    df = df.assign(
        DATE=pd.to_datetime(df['MARKET_DAY']),
        MONTH=lambda d: d['DATE'].dt.month
    )
    long = df.melt(
        ['DATE','MONTH'],
        [f'HE{i}' for i in range(1,25)],
        var_name='HOUR', value_name='PRICE'
    )
    long['PRICE'] = pd.to_numeric(long['PRICE'], errors='coerce')
    long = long.dropna(subset=['PRICE'])
    long['HOUR'] = long['HOUR'].str.extract(r'HE(\d+)').astype(int)

    stats = (
        long
        .groupby(['MONTH','HOUR'])['PRICE']
        .agg(mu='mean', sigma='std')
        .reset_index()
    )
    return stats

In [15]:
rt_stats_wind = monthly_stats(node_RT_wind)
da_stats_wind = monthly_stats(node_DA_wind)

print(rt_stats_wind.head())
print(da_stats_wind.head())

   MONTH  HOUR         mu      sigma
0      1     1  18.216237  27.963367
1      1     2  17.535269  27.760811
2      1     3  15.587849  21.662672
3      1     4  15.502151  24.249682
4      1     5  15.007957  20.960390
   MONTH  HOUR         mu      sigma
0      1     1  20.551398  26.545516
1      1     2  18.314624  24.694891
2      1     3  17.764301  24.430063
3      1     4  17.208387  23.670227
4      1     5  17.863656  23.970794
